In [8]:
import yfinance as yf
from datetime import date, timedelta
import pandas as pd
import numpy as np
import logging
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# List to track failed symbols
failed_symbols = []

def get_historical_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date, interval="1d")
        return data
    except Exception as e:
        logger.error(f"Error downloading data for {symbol}: {e}")
        return None

def calculate_rsi(data, window=14):
    try:
        delta = data["Close"].diff()
        up_moves = delta.clip(lower=0)
        down_moves = -delta.clip(upper=0)
        ema_up = up_moves.ewm(alpha=1/window, min_periods=window-1).mean()
        ema_down = abs(down_moves.ewm(alpha=1/window, min_periods=window-1).mean())
        rs = ema_up / ema_down
        rsi = 100 - (100 / (1 + rs))
        data["RSI"] = rsi
        return data
    except Exception as e:
        logger.error(f"Error calculating RSI for {symbol}: {e}")
        return None

def calculate_indicators(data):
    try:
        data["SMA_5"] = data["Close"].rolling(window=5).mean()
        data["SMA_20"] = data["Close"].rolling(window=20).mean()
        data = calculate_rsi(data)
        return data
    except Exception as e:
        logger.error(f"Error calculating indicators for {symbol}: {e}")
        return None

def identify_trending_stocks(data):
    try:
        condition = np.logical_and(np.logical_and(
            np.logical_and(data['SMA_5'] > data['SMA_20'], data['SMA_5'].shift(1) < data['SMA_20'].shift(1)),
            data['RSI'] < 50),data['Volume'] > 25000)
        condition_1 = np.logical_and(data['SMA_5'] > data['SMA_20'], data['SMA_5'].shift(1) < data['SMA_20'].shift(1))
        data['signal'] = np.where(condition_1, 1, 0)
        data = data[data['signal'] == 1]
        return data
    except Exception as e:
        logger.error(f"Error identifying trending stocks for {symbol}: {e}")
        return None
    
def process_symbol(symbol, symbols_left):
    try:
        start_date = today - timedelta(days=365)
        end_date = today

        logger.info(f"Processing {symbol}... {symbols_left} symbols left.")
        
        start_time = time.time()
        print(symbol)
        data = get_historical_data(symbol, start_date, end_date)
        data['symbol'] = symbol
        if data is not None:
            data = calculate_indicators(data)
            if data is not None:
                data = identify_trending_stocks(data)
                elapsed_time = time.time() - start_time

                logger.info(f"Processed {symbol} in {elapsed_time:.2f} seconds.")
                return data
    except Exception as e:
        logger.error(f"Error processing {symbol}: {e}")
        return None


def send_email(subject, body, attachment_path):
    # Set up the email configuration
    sender_email = "adityachakilam@gmail.com"
    receiver_email = "omprakashchakilam@gmail.com"
    password = "hysz xjad ddyz ilkm"

    # Create the email message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    # Attach the filtered data CSV file
    attachment = open(attachment_path, "rb")
    part = MIMEBase("application", "octet-stream")
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", "attachment; filename=processed_data_filtered.csv")
    message.attach(part)

    # Connect to the SMTP server and send the email
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        print(f"Email sent successfully from {sender_email} to {receiver_email}")

In [9]:
if __name__ == '__main__':
    try:
        start_time_all = time.time()

        # Read stock symbols from CSV file
        symbols = pd.read_csv("nse.csv")["SYMBOL"].tolist()

        # Define start date for backtesting
        today = date.today()



        all_data_list = []
        symbols_left = len(symbols)
        for symbol in symbols:
            data = process_symbol(symbol, symbols_left)
            if data is not None:
                all_data_list.append(data)
            else:
                failed_symbols.append(symbol)
            symbols_left -= 1

        # Concatenate the results into a single DataFrame
        all_data = pd.concat(all_data_list, ignore_index=False)
        all_data = all_data.reset_index()
        # Filter data for the last day
        filtere_date = today - timedelta(days=2)
        # Assuming filtere_date is a date object
        filtere_date = datetime.combine(filtere_date, datetime.min.time())

        # Filtering data based on the condition
        filtered_data = all_data[all_data['Date'] > filtere_date]
        filtered_data[(filtered_data['RSI']< 60) & (filtered_data['Volume']> 25000) & (filtered_data['Open']< filtered_data['Close'])]
        # Save filtered data to CSV
        filtered_data.to_csv("processed_data_filtered.csv")

        # Email subject and body
        subject = "Your Daily Stock Report"
        body = "Attached is the processed data for the last day."

        # Send the email with the filtered data attached
        send_email(subject, body, "processed_data_filtered.csv")

        # Log total runtime
        total_elapsed_time = time.time() - start_time_all
        logger.info(f"Total runtime: {total_elapsed_time:.2f} seconds.")

        # Log failed symbols
        logger.info(f"Failed symbols: {failed_symbols}")

    except Exception as e:
        logger.error(f"An error occurred: {e}")

INFO:__main__:Processing 20MICRONS.NS... 1933 symbols left.


20MICRONS.NS
[*********************100%***********************]  1 of 1 completed

INFO:__main__:Processed 20MICRONS.NS in 0.01 seconds.
INFO:__main__:Processing 21STCENMGM.NS... 1932 symbols left.




1 Failed download:
- 20MICRONS.NS: No data found for this date range, symbol may be delisted
21STCENMGM.NS
[*********************100%***********************]  1 of 1 completed

INFO:__main__:Processed 21STCENMGM.NS in 0.01 seconds.
INFO:__main__:Processing 360ONE.NS... 1931 symbols left.


...



1 Failed download:
- IEX.NS: No data found for this date range, symbol may be delisted
IFBAGRO.NS
[*********************100%***********************]  1 of 1 completed

INFO:__main__:Processed IFBAGRO.NS in 0.02 seconds.
INFO:__main__:Processing IFBIND.NS... 1188 symbols left.




1 Failed download:
- IFBAGRO.NS: No data found for this date range, symbol may be delisted
IFBIND.NS
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- IFBIND.NS: No data found for this date range, symbol may be delisted


INFO:__main__:Processed IFBIND.NS in 0.02 seconds.


KeyboardInterrupt: 